# Regression and Other Stories: Bayesian $R^2$

Bayesian R2. See Chapter 11 in Regression and Other Stories.

See also - Andrew Gelman, Ben Goodrich, Jonah Gabry, and Aki Vehtari (2018). R-squared for Bayesian regression models. The American Statistician, 73:307-209 doi:10.1080/00031305.2018.1549100.

----

### Load packages

In [1]:
using StatsPlots, MCMCChains, Turing, DataFrames, GLM

### Function for Bayesian R-squared for turing models

In [2]:
# TODO
#function bayes_R2(fit)


## Toy data with n=5

In [3]:
x = Array(1:5) .- 3
y = [1.7, 2.6, 2.5, 4.4, 3.8] .- 3
xy = DataFrame(Dict("x"=>x,"y"=>y));

### Lsq fit

In [4]:
fit = lm(@formula(y ~ x), xy)
ols_coef = coef(fit)
yhat = ols_coef[1] .+ ols_coef[2] * x
r = y - yhat
rsq_1 = var(yhat)/var(y)
rsq_2 = var(yhat)/(var(yhat) + var(r))
println(round(rsq_1, digits=3), " ", round(rsq_1, digits=3))

0.766 0.766


### Bayes fit

In [8]:
@model function linear(x,y)
    σ ~ Exponential(1)
    α ~ Normal(0,0.2)
    β ~ Normal(1,0.2)
    μ = α .+ β*x
    y ~ MvNormal(μ,σ)
end; 

In [9]:
model1 = linear(xy.x, xy.y)
fit_bayes = sample(model1, NUTS(), 4000)
summarystats(fit_bayes)

┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference C:\Users\tburch\.julia\packages\Turing\pwMQO\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           α   -0.0017    0.1664     0.0026    0.0023   4850.1634    0.9998    ⋯
           β    0.8149    0.1579     0.0025    0.0031   2984.3672    1.0000    ⋯
           σ    0.7316    0.2982     0.0047    0.0062   2101.1172    0.9998    ⋯
                                                                1 column omitted


In [21]:
posterior = get(fit_bayes, [:α, :β])
post_means = [mean(posterior.α.data), mean(posterior.β.data)];

### Median Bayesian R^2